### Question 1

### >> Before you proceed:

You could just work with the included `account.py` file, and skip this Notebook entirely if you wish to do so. This Notebook is just a repetition of the `account.py` file, and is laid out in a step-by-step way to make it easy for you to get started with.

### Since you've decided to continue ...

### >> STEP 0:

### Take a deep breath! Getting some things ready for you. Just run the cell below, and sit back and relax!

In [30]:
import utils
utils.setup()

Setting up a few accounts to get you started...

User ID generated for user. UserID is:1016. Filling in the user details as provided.
User ID generated for user. UserID is:1017. Filling in the user details as provided.
User ID generated for user. UserID is:1018. Filling in the user details as provided.
Account Created Successfully!! Account number is: 330000017 with available balance: 1500
Account Created Successfully!! Account number is: 330000018 with available balance: 1500
Account Created Successfully!! Account number is: 330000019 with available balance: 1500


###  ------- YOUR WORK STARTS HERE -------

### >> STEP 1:

### Please complete the appropriate methods (`deposit`, `withdraw`, `transfer_funds`) in the `Account` class given below.

In [31]:
import wallet
import csv
import os
import pandas as pd

TRANSACTION_RECORDS = 'account_transactions.csv'
USER_RECORDS = 'user_records.csv'


class Account:

    ''' This class provides multiple functionalities for a pre-existing user account. A UserID, and a transaction account is first needed to be able use this 
    class. The User ID can be generated from the Customer class, and the transacion account can be created using the NewAccount class.

    Usage:
        uid = 1001
        account_number = 3330000407 (optional)
        myaccount = Account(uid,account_number)

    Returns:
        An account object with functions to view details, deposit, withdraw, and transfer money.

    Example 1 (passing both UID and Account number):

        myaccount = Account(1001, 3330000407)

        myaccount.deposit(100)
        > Deposits money (Rs. 100) to account number 1001.

        myaccount.withdraw(50)
        > Withdraws money (Rs. 50) from account number 1001.


    Example 2 (without passing in the Account number):

        youraccount = newAccount(1002)

        youraccount.deposit(100)
        youraccount.view_balance()
        youraccount.view_account_details()

    '''

    _attributes = ("UID", "Account Number", "Balance")

    def __init__(self, usr_id, acc_number=None):

        self.usr_id = usr_id
        self.acc_number = acc_number
        if self.acc_number is None:
            self.acc_number = self._get_account_number()

    def _get_account_number(self, uid=None):

        if not uid:
            uid = self.usr_id
        with open(TRANSACTION_RECORDS, 'r') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                if row:
                    if int(row[0]) == int(uid):
                        return row[1]
        print("Account not found. Please create a new account using the newAccount option.")

    def view_account_details(self):

        with open(TRANSACTION_RECORDS, 'r') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                if row:
                    if int(row[0]) == int(self.usr_id):
                        details = dict(zip(self._attributes[1:], row[1:]))

        for keys, values in details.items():
            print("{} : {}".format(keys, values))

    def view_balance(self, uid=None):

        if not uid:
            uid = self.usr_id

        with open(TRANSACTION_RECORDS, 'r') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                if row:
                    if int(row[0]) == int(uid):
                        return row[2]
        print("Error occurred. Please contact support.")


    def deposit(self, amount):
        try:
            # Check if the account file exists
            if not os.path.exists("account_transactions.csv"):
                print("Transaction records are empty. Please create a new account using the newAccount option.")
                return

            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv("account_transactions.csv")

            # Find the row that matches the user's ID
            user_row = df.loc[df['UID'] == self.usr_id]

            # Check if the user's account exists
            if len(user_row) == 0:
                print("Account not found. Please create a new account using the newAccount option.")
                return

            # Get the current balance from the user's row
            current_balance = int(user_row['Balance'])

            # Add the deposit amount to the current balance
            updated_balance = current_balance + amount

            # Update the balance in the user's row
            df.loc[df['UID'] == self.usr_id, 'Balance'] = updated_balance

            # Write the updated DataFrame to the CSV file
            df.to_csv("account_transactions.csv", index=False)

            print(f"Deposited: Rs. {amount}")
            print(f"Updated Balance: Rs. {updated_balance}")


        # Handle file not found and invalid value errors
        except FileNotFoundError:
            print("Error: File not found.")
        except ValueError:
            print("Error: Invalid value encountered.")

          
    def withdraw(self, amount):
        try:
            # Check if the account file exists
            if not os.path.exists("account_transactions.csv"):
                print("Transaction records are empty. Please create a new account using the newAccount option.")
                return

            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv("account_transactions.csv")

            # Find the row that matches the user's ID
            user_row = None
            for index, row in df.iterrows():
                if row['UID'] == self.usr_id:
                    user_row = row
                    break

            # Check if the user's account exists
            if user_row is None:
                print("Account not found. Please create a new account using the newAccount option.")
                return

            # Get the current balance from the user's row
            current_balance = int(user_row['Balance'])

            # Check if the withdrawal amount is less than or equal to the current balance
            if amount <= current_balance:
                # Subtract the withdrawal amount from the current balance
                updated_balance = current_balance - amount

                # Update the balance in the user's row
                df.loc[index, 'Balance'] = updated_balance

                # Write the updated DataFrame to the CSV file
                df.to_csv("account_transactions.csv", index=False)

                print(f"Withdrawn: Rs. {amount}")
                print(f"Updated Balance: Rs. {updated_balance}")
            else:
                print("Insufficient balance.")

        # Handle file not found and invalid value errors
        except FileNotFoundError:
            print("Error: File not found.")
        except ValueError:
            print("Error: Invalid value encountered.")



    def transfer_funds(self, uid, amount):
        try:
            # Read the CSV file as a pandas DataFrame
            df = pd.read_csv("account_transactions.csv")

            # Check if the DataFrame is empty
            if df.empty:
                print("Transaction records are empty. Please create a new account using the newAccount option.")
                return

            # Get the row for the from account and update its balance
            from_account = df.loc[df['UID'] == self.usr_id]
            if from_account.empty:
                print("Account not found. Please create a new account using the newAccount option.")
                return
            if amount > from_account['Balance'].values[0]:
                print("Insufficient balance.")
                return
            updated_from_balance = from_account['Balance'].values[0] - amount
            df.loc[df['UID'] == self.usr_id, 'Balance'] = updated_from_balance

            # Get the row for the to account and update its balance
            to_account = df.loc[df['UID'] == uid]
            if to_account.empty:
                print("To-account not found.")
                return
            updated_to_balance = to_account['Balance'].values[0] + amount#values[0]is used to index position 0
            df.loc[df['UID'] == uid, 'Balance'] = updated_to_balance

            # Write the updated DataFrame back to the CSV file
            df.to_csv("account_transactions.csv", index=False)

            # Print the transfer details
            print(f"Transferred: Rs. {amount} from account number {self.usr_id} to UID {uid}")
            print(f"Updated Balance for UID {self.usr_id}: Rs. {updated_from_balance}")
            print(f"Updated Balance for UID {uid}: Rs. {updated_to_balance}")

        # Handle file not found and invalid value errors
        except FileNotFoundError:
            print("Error: File not found.")
        except ValueError:
            print("Error: Invalid value encountered.")


Now that you've completed the main part, let us see if everything works as intended!!

### >> STEP 2: 

### Create simple objects and test your methods to verify if they work as desired. 

In [32]:
test_user = wallet.Customer(1001)

In [33]:
test_user.view_profile()

Name    : Daniel Shrestha
DOB     : 02/07/2002
Address : Pulchowk
Phone   : 9841400050
Email   : d.shrestha@gmail.com


In [34]:
test_user.usr_id

1001

In [35]:
test_account = Account(test_user.usr_id)

In [36]:
test_account.acc_number

'330000002'

In [37]:
test_account.view_account_details()

Account Number : 330000002
Balance : 1453


In [38]:
test_account.view_balance()

'1453'

#### Checking the methods that you've implemented ...

In [39]:
test_account.deposit(700)  # This method should deposit Rs 700 to test_account once you complete your code

Deposited: Rs. 700
Updated Balance: Rs. 2153


In [40]:
test_account.withdraw(350) # This method should withdraw Rs 350 from test_account once you complete your code

Withdrawn: Rs. 350
Updated Balance: Rs. 1803


In [12]:
test_account.transfer_funds(1007, 747) # This method should transfer Rs 747 from test_account to user account with UID: 1003 once you complete your code

Transferred: Rs. 747 from account number 1005 to UID 1007
Updated Balance for UID 1005: Rs. 1365
Updated Balance for UID 1007: Rs. 4205


### >> STEP 3:

#### Once you complete the methods in STEP 1, you can copy and paste the entire content of the above cell (STEP 1 only) to the included `account.py` python file (replacing the already existing content of the `account.py` script entirely).

### >> STEP 4:

### Now import and run the `runtest ` method from `utils.py` file to see if everything works as intended (as shown below).

In [154]:
import utils
utils.runtest()

Starting the TEST by registering a new user and creating user profile.

User ID generated for user. UserID is:1010. Filling in the user details as provided.

Viewing User profile details:

Name    : Jenny Wallace
DOB     : 03/09/2003
Address : Oldenberg
Phone   : 9841420082
Email   : j.wallace@gmail.com

Updating the user Address to :'Baneshwor-KTM'
User(UID: 1010)'s Address updated successfully to Baneshwor-KTM!

Viewing user profile details after updating address:

Name    : Jenny Wallace
DOB     : 03/09/2003
Address : Baneshwor-KTM
Phone   : 9841420082
Email   : j.wallace@gmail.com

Updating the user phone number to :9851011121
User(UID: 1010)'s Phone updated successfully to 9851011121!

Viewing user profile details after updating phone number:

Name    : Jenny Wallace
DOB     : 03/09/2003
Address : Baneshwor-KTM
Phone   : 9851011121
Email   : j.wallace@gmail.com

Proceeding to create a new transaction account for our user with Rs. 1500 starting balance.

Account Created Successfull

### If everything runs fine, you should see a message along the lines of : `Preliminary tests ran successfully!!`

###  ------- END OF TASK 1 -------